# Nats (Neural Autonomic Transport System)
##  NATS (Core)
NATS is a high-performance, distributed messaging system that facilitates communication between applications through a publish/subscribe pattern. 

- Messaging System: NATS is a publish/subscribe messaging system where applications publish messages to subjects (similar to topics) and subscribers can receive messages from those subjects. 
- Fast & Scalable: Designed for high performance and scalability, NATS is suitable for microservices, edge computing, and IoT applications. 
- Simple Design: NATS employs a simple design where clients publish and subscribe to subjects, and the server handles routing messages. 
- Interoperability:Core NATS and JetStream work together; JetStream streams listen to NATS messages. 


## NATS JetStream:

NATS JetStream is a persistence engine built into NATS that enables message storage and replay, offering features like streaming, at-least-once delivery, and historical data replay

- Persistence Engine: JetStream is a built-in persistence engine within NATS that allows messages to be stored and replayed. 
- Streaming & Replay: JetStream enables messages to be captured and replayed to consumers as needed, even if they weren't actively subscribed at the time of publication. 
- At-Least-Once Semantics: JetStream provides at-least-once delivery guarantees, ensuring messages are delivered at least once to consumers. 
- Features: Includes streaming, message queueing, durable subscriptions, and historical data replay. 
- Data Storage: JetStream stores messages in streams, which define how messages are stored and limits such as retention policies. 
- Consumers: Consumers are responsible for replaying messages from a stream, and their configuration specifies how messages should be presented to clients. 
- Object Store: JetStream also includes an object store feature for storing and transferring files. 
- Scalability: JetStream supports horizontal scaling and can be deployed in various environments, including edge and cloud.

In essence, NATS is the core messaging system, while NATS JetStream adds the persistence and streaming capabilities, making it suitable for more complex scenarios where message storage and replay are required


## Installation

- https://docs.nats.io/running-a-nats-service/introduction/installation
- https://github.com/nats-io/nats.docs/blob/master/running-a-nats-service/installation.md
- https://github.com/nats-io/nats-server

![image.png](nats.png)

In [10]:
### docker run -p 4222:4222 nats -js
import asyncio
import nats
from nats.errors import ConnectionClosedError, TimeoutError, NoServersError

import os
import glob
import json
import uuid
import nats
import mimetypes

In [11]:
# nc = await nats.connect("localhost")
# print("Maximum payload is %d bytes" % nc.max_payloa

In [12]:


async def publish_images_from_folder(nats_url, stream_name, subject, input_folder):
    """
    Publishes metadata for all images from a folder to NATS JetStream.
    Instead of sending binary data, it sends metadata with URI.
    """
    try:
        nc = await nats.connect(nats_url)
        js = nc.jetstream()

        for file_path in glob.glob(os.path.join(input_folder, "*")):
            if os.path.isfile(file_path):
                filename = os.path.basename(file_path)
                # Get file size
                file_size = os.path.getsize(file_path)
                # Get file mime type
                mime_type, _ = mimetypes.guess_type(file_path)
                if not mime_type:
                    mime_type = "application/octet-stream"  # Default if can't determine
                
                # Create absolute path for URI
                absolute_path = os.path.abspath(file_path)
                print(absolute_path)
                # Create message with metadata
                message = {
                    "version": "1.0",
                    "batchId": str(uuid.uuid4()),
                    "source": {
                        "uri": f"file://{absolute_path}",
                        "mimeType": mime_type,
                        "size": file_size
                    },
                    "ocrOptions": {
                        "languages": ["en"],
                        "preProcessing": ["resize", "2x-zoom", "rgb"]
                    },
                    "state": {
                        "fileId": str(uuid.uuid4()).replace("-", "_")[:20],
                        "scanId": "sony-doc"
                    }
                }
                
                # Convert to JSON and encode to bytes for NATS
                message_data = json.dumps(message).encode()
                
                await js.publish(subject, message_data, stream=stream_name, headers={"file-name": filename})
                print(f"Published metadata for '{filename}' to {subject} in stream {stream_name}")
    except Exception as e:
        print(f"Error publishing metadata: {e}")
    finally:
        await nc.close()


In [13]:
nats_url = "nats://localhost:4222"
stream_name = "IMAGE_STREAM"
subject = "images.process"
input_folder = "docs" #Replace with your file path





In [14]:
os.getcwd()

'/mnt/d/repos2/nats'

In [15]:

await publish_images_from_folder(nats_url, stream_name, subject, input_folder)
        

/mnt/d/repos2/nats/docs/base64.html
Published metadata for 'base64.html' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/contribution_form_Juan_23082023.pdf
Published metadata for 'contribution_form_Juan_23082023.pdf' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/image_0.png
Published metadata for 'image_0.png' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/image_1.png
Published metadata for 'image_1.png' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/image_2.png
Published metadata for 'image_2.png' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/pict001.jpg
Published metadata for 'pict001.jpg' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/pict002.jpg
Published metadata for 'pict002.jpg' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/pict003.jpg
Published metadata for 'pict003.jpg' to images.process in stream IMAGE_STREAM
/mnt/d/repos2/nats/docs/ppt_test.ppt
Pub